In [16]:
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.datasets import mnist
from keras import layers
from tensorflow.keras.optimizers import Adam
import keras
import tensorflow as tf

# Build the DCGAN model

### Discriminator and Generator heads builder

In [20]:
def dcgan_heads_builder(input_shape, coef_latent = 4, show_summary = True):
    """Build a simple DCGAN model for low resolution image
     generation tasks"""
    def build_discriminator(input_shape, show_summary = True):
        """Builds the discriminator head, which downsamples the input and
           will be us used to classify fake/true data"""
        base_channels =  input_shape[0]
        latent_dim = base_channels * 2
        discriminator = keras.Sequential([
            layers.Input(shape=input_shape),
            layers.Conv2D(base_channels, kernel_size=4, strides=2, padding="same"),
            layers.LeakyReLU(negative_slope=0.2),
            layers.Conv2D(base_channels * 2, kernel_size=4, strides=2, padding="same"),
            layers.LeakyReLU(negative_slope=0.2),
            layers.Conv2D(base_channels * 2, kernel_size=4, strides=2, padding="same"),
            layers.LeakyReLU(negative_slope=0.2),
            layers.Flatten(),
            layers.Dropout(0.2),
            layers.Dense(1, activation="sigmoid")
        ], name="discriminator")

        if show_summary:
            discriminator.summary()

        return discriminator, latent_dim
    
    def build_generator(latent_dim, coef_latent, show_summary = True):
        """Builds the generator head which will accept random noise(shape = latent_dim * coef_latent^2 dimensionality)
           and upscale it to a new synthetic image
           
           Feel free to adjust the coefficient according to your datasets needs. For example, 
           4 * 4 will generate images of (32,32,3) shape.

           """
        generator = keras.Sequential([
            keras.Input(shape=(latent_dim,)),
            layers.Dense(coef_latent * coef_latent * latent_dim),
            layers.Reshape((coef_latent, coef_latent, latent_dim)),
            layers.Conv2DTranspose(latent_dim, kernel_size=4, strides=2, padding="same"),
            layers.LeakyReLU(negative_slope=0.2),
            layers.Conv2DTranspose(latent_dim * 2, kernel_size=4, strides=2, padding="same"),
            layers.LeakyReLU(negative_slope=0.2),
            layers.Conv2DTranspose(latent_dim * 4, kernel_size=4, strides=2, padding="same"),
            layers.LeakyReLU(negative_slope=0.2),
            layers.Conv2D(3, kernel_size=5, padding="same", activation="sigmoid")],
            name="generator")
        
        if show_summary:
            generator.summary()

        return generator
    
    # Create the two heads as Keras sequential models
    discriminator, latent_dim = build_discriminator(input_shape, show_summary = show_summary)
    generator = build_generator(latent_dim, coef_latent=coef_latent, show_summary = show_summary)

    return generator, discriminator, latent_dim

### DCGAN Model

In [23]:

class DCGAN(keras.Model):
    def __init__(self, discriminator, generator, latent_dim):
        super().__init__()
        self.discriminator = discriminator
        self.generator = generator
        self.latent_dim = latent_dim
        self.seed_generator = tf.random.Generator.from_seed(1337)
        self.name = 'DCGAN'

    def compile(self, d_optimizer, g_optimizer, loss_fn):
        super().compile()
        self.d_optimizer = d_optimizer
        self.g_optimizer = g_optimizer
        self.loss_fn = loss_fn
        self.d_loss_metric = keras.metrics.Mean(name="d_loss")
        self.g_loss_metric = keras.metrics.Mean(name="g_loss")

    @property
    def metrics(self):
        return [self.d_loss_metric, self.g_loss_metric]

    def train_step(self, real_images):
        # Sample random points in the latent space
        batch_size = tf.shape(real_images)[0]
        random_latent_vectors = self.seed_generator.normal(
            shape=(batch_size, self.latent_dim)
        )

        # Decode them to fake images
        generated_images = self.generator(random_latent_vectors)

        # Combine them with real images
        combined_images = tf.concat([generated_images, real_images], axis=0)

        # Create labels for generated and fake images
        labels = tf.concat(
            [tf.ones((batch_size, 1)), tf.zeros((batch_size, 1))], axis=0
        )
        # Force the discriminator to struggle by adding some random noise
        # to the labels.
        labels += 0.05 * tf.random.uniform(tf.shape(labels))

        # Train the discriminator
        with tf.GradientTape() as tape:
            predictions = self.discriminator(combined_images)
            d_loss = self.loss_fn(labels, predictions)
        grads = tape.gradient(d_loss, self.discriminator.trainable_weights)
        self.d_optimizer.apply_gradients(
            zip(grads, self.discriminator.trainable_weights)
        )

        # Sample random points in the latent space
        random_latent_vectors = self.seed_generator.normal(
            shape=(batch_size, self.latent_dim)
        )

        # Assemble labels that say "all real images"
        misleading_labels = tf.zeros((batch_size, 1))

        # Train the generator (note that we should *not* update the weights
        # of the discriminator)!
        with tf.GradientTape() as tape:
            predictions = self.discriminator(self.generator(random_latent_vectors))
            g_loss = self.loss_fn(misleading_labels, predictions)
        grads = tape.gradient(g_loss, self.generator.trainable_weights)
        self.g_optimizer.apply_gradients(zip(grads, self.generator.trainable_weights))

        # Update metrics
        self.d_loss_metric.update_state(d_loss)
        self.g_loss_metric.update_state(g_loss)
        return {
            "d_loss": self.d_loss_metric.result(),
            "g_loss": self.g_loss_metric.result(),
        }


IMAGE_SHAPE = (32,32,3)
generator, discriminator, latent_dim = dcgan_heads_builder(IMAGE_SHAPE, show_summary= False)
model = DCGAN(discriminator=discriminator, generator=generator, latent_dim=latent_dim)
model.compile(
    d_optimizer=keras.optimizers.Adam(learning_rate=0.0002, beta_1=0.5),
    g_optimizer=keras.optimizers.Adam(learning_rate=0.0002, beta_1=0.5),
    loss_fn=keras.losses.BinaryCrossentropy(from_logits=True)
)

model.summary()

Model: "DCGAN"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ discriminator (Sequential)      │ ?                      │       101,025 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ generator (Sequential)          │ ?                      │       807,107 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 908,132 (3.46 MB)

 Trainable params: 908,132 (3.46 MB)

 Non-trainable params: 0 (0.00 B)